# 1. Data Exploration & Cleaning

## Import dependencies, set up environment

1. `python3 -m venv .venv`
2. utilize virtual environment
    - (LINUX/MAC) `source .venv/bin/activate`
    - (WINDOWS) `.venv\Scripts\Activate.    ps1`
3. `pip install -r requirements.txt`

In [1]:
# Jupyter magic
%run ../util/dependencies.py

## Planetary Systems

Upon reviewing the quality of data from the "Planetary Systems" (PS) database, it was deemed better to pivot towards the "Planetary Systems Composite Data" (PSCompPars) database. In brief:
1. PS details a record for each exoplanet and each one of its references (this helps us reach original literature analyses of these bodies). Missing data is prevelant.
2. PSCompPars curates a “best available” or “most complete” set of parameters for each planet, pulling from multiple references.

So far as our exploration of exoplanets and their stars (studying the whole population of exoplanets thus far), this seems outside the scope of our analysis, and creates a cumbersome process of exploring the data. This will aid in limiting time spent cleaning the dataset, and limit our analysis to 6065 from a daunting ~32,000 records.

For an explanation on how the composite dataset aggregates all available information on exoplanet figures, please see <https://exoplanetarchive.ipac.caltech.edu/docs/pscp_calc.html>.

In [5]:
# TAP base URL (Planetary Systems Composite Data)
url = "https://exoplanetarchive.ipac.caltech.edu/TAP/sync"
# The "Planetary Systems Composite Data" database (confirmed exoplanets) is encoded as "PSCompPars" within the Exoplanet Archive
ADQL_query =    "SELECT " \
                "pl_controv_flag, pl_name, hostname, pl_letter, sy_snum, discoverymethod, disc_year," \
                "pl_radj, pl_massj, st_spectype, st_rad, st_mass, st_met, st_lum, st_teff, st_rad " \
                "FROM PSCompPars"

# Request data as CSV
params = {
    "query": ADQL_query,
    "format": "csv"
}
response = requests.get(url, params=params)

# Load into "Planetary Systems" DataFrame
pscp = pd.read_csv(io.StringIO(response.text))
print("Data loaded successfully. Number of records:", len(pscp))

# Save to parquet for local use and following notebooks
# (parquets are smaller files than csv, so good as an intermediate file type for future processing)
file_name = 'pscp_01_raw.csv'

try:
    pscp.to_csv('../data/' + file_name, index=False)
    print('Data saved as \'' + file_name + '\'')
except Exception as e:
    print('Data failed to save as \'' + file_name + '\': ' + e)

Data loaded successfully. Number of records: 6065
Data saved as 'pscp_01_raw.csv'


Over 355 columns in this dataset!! All different features we can analyze in another related project pertaining to exoplanet exploration and methods for doing so . . .

After reviewing the column descriptions (as defined here: <https://exoplanetarchive.ipac.caltech.edu/docs/API_PS_columns.html>), the following features are relevant to our exploration:
1. pl_controv_flag (is the comfirmation of this planet questioned?)
2. pl_name
3. hostname (most common star name)
4. pl_letter
5. sy_snum
6. discoverymethod
7. disc_year
8. pl_radj
9. pl_massj
10. st_spectype
11. st_rad
12. st_mass
13. st_met
14. st_lum
15. st_rad

As we will see it later, it is important to get an overview of all of the stars we have seen in the observable universe thus far, to draw a picture of what exoplanet host (stellar) date is accessable to us, vs the stars that are in the observable universe. This is done as a means to detect bias from:
1. **Educated Assumptions** Stars we choose to observe,
2. **Technical Limitations** Stars are easier to observe, and
3. **Physical Stellar Characteristics** Stars that tend to have more planets

As such, the following dataset was appended to the 

## Kepler Stellar
Observe the data from the kepler mission, designed to identify exoplanets in our observable universe.

In [6]:
# TAP base URL (Kepler Stellar Table)
url = "https://exoplanetarchive.ipac.caltech.edu/TAP/sync"
# The "Kepler Stellar Table" database (confirmed exoplanets) is encoded as "keplerstellar" within the Exoplanet Archive
ADQL_query =    "SELECT " \
                "kepid, tm_designation, teff, feh, radius, mass, dens, " \
                "nconfp, nkoi, ntce " \
                "FROM keplerstellar"

# Request data as CSV
params = {
    "query": ADQL_query,
    "format": "csv"
}
response = requests.get(url, params=params)

# Load into "Kepler Stellar" DataFrame
ks = pd.read_csv(io.StringIO(response.text))
print("Data loaded successfully. Number of records:", len(ks))

# Save raw data as csv
file_name = 'ks_01_raw.csv'

try:
    ks.to_csv('../data/' + file_name, index=False)
    print('Data saved as \'' + file_name + '\'')
except Exception as e:
    print('Data failed to save as \'' + file_name + '\': ' + e)

Data loaded successfully. Number of records: 990244
Data saved as 'ks_01_raw.csv'


## Gaia DR3
This dataset is utilized to determine the makeup of stars in the (known) universe. While the distribution of star classifications has already been done (and referenced in the README), this data was explored for the following reasons:
1. Practice utilizing APIs in a python context
2. Experience utilizing the the Gaia DR3 dataset: data I am sure to revisit in future astro data analytic endevours
3. Verifying 3rd party figures are accurate

In [2]:
from astroquery.gaia import Gaia

# Queary for retrieving the stellar "classification" data
# This query utilizes random_index to achieve an unbiased representation of the population of observed stars
OLD_ADQL_QUERY = """
SELECT
    gs.source_id,
    gs.teff_gspphot,
    gs.bp_rp,
    gs.random_index,
    ap.radius_gspphot,
    ap.mass_flame
FROM gaiadr3.gaia_source AS gs
INNER JOIN gaiadr3.astrophysical_parameters as ap
ON gs.source_id = ap.source_id
WHERE
    gs.teff_gspphot IS NOT NULL
    AND gs.parallax > 0
    AND gs.random_index < 20000
"""

# made a mistake of limiting to 4806 records with INNER JOIN, thus . . .
ADQL_QUERY = """
SELECT
    gs.source_id,
    gs.teff_gspphot,
    gs.bp_rp,
    gs.random_index,
    ap.radius_gspphot,
    ap.mass_flame
FROM gaiadr3.gaia_source AS gs
LEFT JOIN gaiadr3.astrophysical_parameters as ap
ON gs.source_id = ap.source_id
WHERE
    gs.parallax > 0
    AND gs.random_index < 20000000 --twenty million stars!!
"""

# Run the query
Gaia.MAIN_GAIA_TABLE = "gaiadr3.gaia_source"
Gaia.ROW_LIMIT = -1  # no limit
my_job_query = Gaia.launch_job_async(ADQL_QUERY) # async for long queries?!!?
gaia_df = my_job_query.get_results().to_pandas()

# Save in CSV file for further analysis in '02_clean_and\ _explore_gaiadr3.ipynb'
file_name = 'gaiadr3_01_raw'
gaia_df.to_csv('../data/' + file_name + '.csv', index_label=False) # accidentally included indeces originally :D

INFO: Query finished. [astroquery.utils.tap.core]


### 3 mil Cap, is this sample sufficient?
Turns out that I needed to rewrite the following line: "my_job_query = Gaia.launch_job_async(ADQL_QUERY)", as Gaia limits (on the server side) query responses to 2000 records for synced queries! Good to know . . .

It seems like there is a row limit of 3 million, I tried to fix this by having a ROW_LIMIT of -1, but ultimately I will be limited by what the server allows. Further investigation sees that I have hit the threshold of 3 million returned records by Gaia's servers, and unfortunatly this is a hard cap to avoid potential issues with overloading their servers.

Alternatively, I could work around this feature by doing chunks of calls to the server, and then combining this data into a single dataframe. However, for my purposes 3 million is LUDICROUS and more than enough records for me to effectively analyze star classification distributions here's the math:

### random_index
"random_index" was utilized to ensure that the entire collection of observed stars was to achieve an accurate representation of the population of observed stars,  while not downloading all datapoints in the Gaia DR3 dataset. See below for Gaia's explanation of this feature from their DR3 main source catalogue (https://gea.esac.esa.int/archive/documentation/GDR3/Gaia_archive/chap_datamodel/sec_dm_main_source_catalogue/ssec_dm_gaia_source.html)

"

    random_index : Random index for use when selecting subsets (long)

    A random index which can be used to select smaller subsets of the data that are still representative. The column contains a random permutation of the numbers from 0 to N−1, where N is the number of sources in the table.

    The random index can be useful for validation (testing on 10 different random subsets), visualization (displaying 1% of the data), and statistical exploration of the data, without the need to download all the data.
    
"

### paralax > 0 (preemtive cleaning procedure)
Please see the README documentation on an explanation of parallax if you are unfamiliar.
In a perfect world negative parallaxes should NOT be measured. However, a large sum of stars are measured of having a negative parallax as a consequence of the incredible distance of these stars from Earth. Measurement errors of this nature appear from large distances when Gaia is unable to measure parallax precisely. Removing datapoints with negative parallaxes ensures that stars with unreliable and uniformative distance measurements are not factored into our analysis. This is a practical cut to our data to ensure the quality and integrity of our data is maintained.